In [1]:
import csv
import time
import lyricsgenius as lg
import pandas
import requests
from urllib3.exceptions import ReadTimeoutError

In [20]:
access_token = "xoJrDroACqJHK6S_MAZh5EB1qcLxv46BGlrlEgiUvlVPaLCbGhLckJSGqW9zTG-U"
genius = lg.Genius(
    access_token,
    skip_non_songs=True,
    excluded_terms=["(Remix)", "(Live)"],
    remove_section_headers=True,
)
artists = []
df = pandas.read_csv("artist_Q.csv")

for index, row in df.iloc[11:16].iterrows():
    artists.append(row["name"])

In [23]:
def get_artist_n_lyrics(artist):
  songs = genius.search_artist(artist, max_songs = 5, per_page=2, get_full_info=True, allow_name_change=True, include_features=True)
  return songs

In [25]:
def get_tracks(artist_name):
    try:
        return get_artist_n_lyrics(artist_name)
    except requests.exceptions.Timeout as e:
        print("Request timed out: {}".format(e))
        print("Retrying in 30 seconds...")
        time.sleep(30)
        return get_tracks(artist_name)
    except requests.exceptions.ReadTimeout as e:
        print("Request timed out: {}".format(e))
        print("Retrying in 30 seconds...")
        time.sleep(30)
        return get_tracks(artist_name)
    except ReadTimeoutError as e:
        print("Read timed out: {}".format(e))
        print("Retrying in 30 seconds...")
        time.sleep(30)
        return get_tracks(artist_name)
    except lg.exceptions.SongNotFound as e:
        # handle song not found exception
        print(f"Song not found error: {e}")
        return None
    except lg.exceptions.Timeout as e:
        print("Timeout Error: {e}")
        print("Retrying in 30 secs...")
        time.sleep(30)
        return get_tracks(artist_name)

In [ ]:
songs = get_tracks("Lady Gaga")
songs_list = songs.to_dict()["songs"]

Dictionary of songs_list:
Artist dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'iq', 'description_annotation', 'user', 'songs'])
Dictionary of songs_list["songs"]:
Song dict_keys(['annotation_count', 'api_path', 'artist_names', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'language', 'lyrics_owner_id', 'lyrics_state', 'path', 'pyongs_count', 'relationships_index_url', 'release_date_components', 'release_date_for_display', 'release_date_with_abbreviated_month_for_display', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'featured_artists', 'primary_artist', 'apple_music_id', 'apple_music_player_url', 'description', 'embed_content', 'featured_video', 'lyrics_placeholder_reason', 'recording_location', 'release_date', 'current_user_metadata', 'album', 'custom_performances', 'description_annotation', 'lyrics_marked_complete_by', 'lyrics_marked_staff_approved_by', 'media', 'producer_artists', 'song_relationships', 'translation_songs', 'verified_annotations_by', 'verified_contributors', 'verified_lyrics_by', 'writer_artists', 'artist', 'lyrics'])

In [42]:
songs_list = [song for song in songs_list if (song is not None) & (song["language"] != "null") & (song["language"] == "en") ]
songs_list

[{'annotation_count': 14,
  'api_path': '/songs/1316',
  'artist_names': 'Lady Gaga',
  'full_title': 'Poker Face by\xa0Lady\xa0Gaga',
  'header_image_thumbnail_url': 'https://images.genius.com/c18b7224de0d97d107ae0b2c2aff35dc.300x300x1.png',
  'header_image_url': 'https://images.genius.com/c18b7224de0d97d107ae0b2c2aff35dc.1000x1000x1.png',
  'id': 1316,
  'language': 'en',
  'lyrics_owner_id': 263424,
  'lyrics_state': 'complete',
  'path': '/Lady-gaga-poker-face-lyrics',
  'pyongs_count': 93,
  'relationships_index_url': 'https://genius.com/Lady-gaga-poker-face-sample',
  'release_date_components': {'year': 2008, 'month': 9, 'day': 23},
  'release_date_for_display': 'September 23, 2008',
  'release_date_with_abbreviated_month_for_display': 'Sep. 23, 2008',
  'song_art_image_thumbnail_url': 'https://images.genius.com/6d4915528495453664d61b6e93bf069b.300x300x1.png',
  'song_art_image_url': 'https://images.genius.com/6d4915528495453664d61b6e93bf069b.1000x1000x1.png',
  'stats': {'accept

In [44]:
import json
with open("handled_albums.json", "w", encoding="utf-8") as f:
    json.dump(songs_list, f, ensure_ascii=False)